In [ ]:
#import packages
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
#import pymannkendall as mk
#
import cartopy.crs as ccrs
import time

from functions import *
era_land=xr.open_dataset('eralandmask_regid.nc').rename({'lat':'latitude','lon':'longitude'})

## Compute hotspells

### ERA5

In [ ]:
#Hotspells
era_tmean_full=xr.open_dataset('era5/era5_tmean_daily_1950_2024_regridcon.nc').load()
era_tmean_lon=rotlon_180(era_tmean_full.rename({'lat':'latitude','lon':'longitude'}))
era_tmean_noleap=era_tmean_lon.convert_calendar('noleap')
era_tmean_noleap=era_tmean_noleap.sortby(era_tmean_noleap.time)


# #anomally based heat days
# era_dmaxanom=era_tmean_noleap.groupby('time.dayofyear')-era_tmean_noleap.groupby('time.dayofyear').mean(dim='time')
outqt_anom=refqtl_single(era_dmaxanom.drop_vars('dayofyear'))
#outqt_anom.to_netcdf('era5_tmean_quantiles_1950_2024_regrid.nc')

thresh_anom=thresh_single(era_dmaxanom.drop_vars('dayofyear'),outqt_anom)
thresh_anom=thresh_anom.sortby(thresh_anom.time)
#thresh_anom.to_netcdf('era5_tmean_thresholds_5dsmooth_1950_2024_regrid.nc')

heat_1_d=heat_day1(thresh_anom,era_dmaxanom,pst=3)
heat_1_d.to_dataset(name='t2m').to_netcdf('era5/era5_tmean_heatdays_1950_2024_regrid.nc')

In [ ]:
#Detrended hotspells
era_tmean_full=xr.open_dataset('era5/era5_tmean_daily_1950_2024_regridcon.nc').load()
era_tmean_lon=rotlon_180(era_tmean_full.rename({'lat':'latitude','lon':'longitude'}))
era_tmean_noleap=era_tmean_lon.convert_calendar('noleap')
era_tmean_noleap=era_tmean_noleap.sortby(era_tmean_noleap.time)
yrs=np.unique(era_tmean_noleap.time.dt.year.values)
dat_5yanom,dat_5yclims=rem_5yclim(era_tmean_noleap.t2m,30,yrs)
era_detanom=dat_5yanom.to_dataset(name='t2m')
outqt_anom=refqtl_single(era_detanom.drop_vars('dayofyear'))
#outqt_anom.to_netcdf('era5_tmean_quantiles_1950_2024_regrid.nc')

thresh_anom=thresh_single(era_detanom.drop_vars('dayofyear'),outqt_anom)
thresh_anom=thresh_anom.sortby(thresh_anom.time)
#thresh_anom.to_netcdf('era5_tmean_thresholds_5dsmooth_1950_2024_regrid.nc')

heat_1_d=heat_day1(thresh_anom,era_detanom,pst=3)
heat_1_d.to_dataset(name='t2m').to_netcdf('era5/era5_detrendedtmean_heatdays_1950_2024_regrid.nc')

### Berkley 

In [ ]:
# Hot spells
#anomally based heat days (1950...)
berkley_tmean_full=xr.open_dataset('berkley_tmean_1950-2021_regridcon.nc').load()
berkley_tmean_full=rotlon_180(berkley_tmean_full.rename({'lat':'latitude','lon':'longitude','temperature':'t2m'}))
bk_tmn_noleap=berkley_tmean_full.convert_calendar('noleap')
bk_tmn_noleap=bk_tmn_noleap.sortby(bk_tmn_noleap.time)
berkley_dmaxanom=bk_tmn_noleap.groupby('time.dayofyear')-bk_tmn_noleap.groupby('time.dayofyear').mean(dim='time')
#berkley_dmaxanom=berkley_dmaxanom.rename({'lat':'latitude','lon':'longitude'})
berkley_dmaxanom=berkley_dmaxanom.sortby(berkley_dmaxanom.time)

outqt_anom=refqtl_single(berkley_dmaxanom.drop_vars('dayofyear'))
#outqt_anom.to_netcdf('berkley_tmean_anom_quantiles_1950_2021_regrid.nc')

thresh_anom=thresh_single(berkley_dmaxanom.drop_vars('dayofyear'),outqt_anom)
thresh_anom=thresh_anom.sortby(thresh_anom.time)
#thresh_anom.to_netcdf('berkley_tmean_anom_thresholds_5dsmooth_1950_2021_regrid.nc')

heat_1_d=heat_day1(thresh_anom,berkley_dmaxanom,pst=3)
heat_1_d.to_dataset(name='t2m').to_netcdf('berkley_tmean_heatdays_1950_2021_regrid.nc')

In [ ]:
# Detrrended hotspells
berkley_tmean_full=xr.open_dataset('berkley_tmean_1950-2021_regridcon.nc').load()
berkley_tmean_full=rotlon_180(berkley_tmean_full.rename({'lat':'latitude','lon':'longitude','temperature':'t2m'}))
bk_tmn_noleap=berkley_tmean_full.convert_calendar('noleap')
bk_tmn_noleap=bk_tmn_noleap.sortby(bk_tmn_noleap.time)
yrs=np.unique(bk_tmn_noleap.time.dt.year.values)
dat_5yanom,dat_5yclims=rem_5yclim(bk_tmn_noleap.t2m,30,yrs)

bk_detanom=dat_5yanom.to_dataset(name='t2m')
#berkley_dmaxanom=berkley_dmaxanom.rename({'lat':'latitude','lon':'longitude'})
#berkley_dmaxanom=berkley_dmaxanom.sortby(berkley_dmaxanom.time)

outqt_anom=refqtl_single(bk_detanom.drop_vars('dayofyear'))
#outqt_anom.to_netcdf('berkley_tmean_anom_quantiles_1950_2021_regrid.nc')

thresh_anom=thresh_single(bk_detanom.drop_vars('dayofyear'),outqt_anom)
thresh_anom=thresh_anom.sortby(thresh_anom.time)
#thresh_anom.to_netcdf('berkley_tmean_anom_thresholds_5dsmooth_1950_2021_regrid.nc')

heat_1_d=heat_day1(thresh_anom,bk_detanom,pst=3)
heat_1_d.to_dataset(name='t2m').to_netcdf('berkley_detrendedtmean_heatdays_1950_2021_regrid.nc')

## CMIP6

### Historical + SSP5 (1950-2021)

In [ ]:
#'ACCESS-CM2','AWI-CM-1-1-MR','BCC-CSM2-MR', 'CanESM5','CESM2','CMCC-ESM2','CNRM-ESM2-1','EC-Earth3-CC','GFDL-ESM4','HadGEM3-GC31-MM''IITM-ESM','INM-CM5-0',
#360day KACE, UKESM
models1=['ACCESS-CM2','AWI-CM-1-1-MR','BCC-CSM2-MR', 'CanESM5','CESM2','CMCC-ESM2','CNRM-ESM2-1','EC-Earth3-CC','GFDL-ESM4','IITM-ESM','INM-CM5-0','KIOST-ESM','MIROC6','MPI-ESM1-2-LR','MRI-ESM2-0','NESM3','NorESM2-MM','TaiESM1']# temp_datcut={}
models22=['CESM2-WACCM', 'CMCC-CM2-SR5', 'CNRM-CM6-1','INM-CM4-8','IPSL-CM6A-LR', 'MIROC-ES2L','CNRM-CM6-1-HR']
models=models1+models22#['ACCESS-CM2','AWI-CM-1-1-MR','BCC-CSM2-MR', 'CanESM5','CESM2', 'CESM2-WACCM','CMCC-ESM2', 'CMCC-CM2-SR5','CNRM-ESM2-1', 'CNRM-CM6-1', 'CNRM-CM6-1-HR','EC-Earth3-CC','GFDL-ESM4','IITM-ESM','INM-CM4-8', 'INM-CM5-0', 'IPSL-CM6A-LR','KIOST-ESM','MIROC6', 'MIROC-ES2L','MPI-ESM1-2-LR','MRI-ESM2-0','NESM3','NorESM2-MM','TaiESM1']
len(models)len(models)

In [ ]:
#concat hist and ssp
for mod in models:
    dat=xr.open_mfdataset('cmip6/hist_ssp/tas_day_'+mod+'*.nc')
    temp_dator[mod]=dat
    dat.to_netcdf('cmip6/hist_ssp/tas_day_'+mod+'_hist_ssp_full.nc')
#     dat=xr.open_dataset('cmip6/hist_ssp/tas_day_'+mod+'_hist_ssp_full_regridcon.nc')
#     print (dat.time.dt.calendar)

In [ ]:
#reread files

to_con=[]
temp_datcut={}
for mod in models:
    dat=xr.open_dataset('cmip6/tmean/tmean_'+mod+'_histssp_regridcon.nc')#.rename({'lat':'latitude','lon':'longitude'})
    print (dat.time.dt.calendar)
    dat1=dat.convert_calendar('noleap').rename({'lon':'longitude','lat':'latitude'})
    temp_dat=rotlon_180(dat1)
    temp_datcut[mod]=temp_dat.sel(time=(slice('1950-01-01','2024-12-31')))


In [ ]:
#compute hotspells

for i,mod in enumerate(models):
    temp=temp_datcut[mod].tas.to_dataset(name='t2m')
    dmaxanom=temp.groupby('time.dayofyear')-temp.groupby('time.dayofyear').mean(dim='time')
    dmaxanom=dmaxanom.sortby(dmaxanom.time)
    #dmaxanom=cuttimestart(dmaxanom,1979)
    outqt_anom=refqtl_single(dmaxanom.drop_vars('dayofyear'))
    #outqt_anom.to_netcdf('cmip6/'+mod+'_anom_quantiles_15dsmooth_1950_2023_1x1.nc')

    thresh_anom=thresh_single(dmaxanom.drop_vars('dayofyear'),outqt_anom)
    thresh_anom=thresh_anom.sortby(thresh_anom.time)
    #thresh_anom.to_netcdf('cmip6/'+mod+'_anom_thresholds_5dsmooth_1950_2023_1x1.nc')

    heat_1_d=heat_day1(thresh_anom,dmaxanom,pst=3)
    heat_1_d.to_dataset(name='htdays').to_netcdf('cmip6/heat/'+models_out[i]+'_tmean_heatdays_hist_ssp5_1950_2024_regridcon.nc')

### Future SSP5 (2025-2100)

In [ ]:
#reread files (future)
to_con=[]
temp_datcut={}
for mod in models:
    dat=xr.open_dataset('cmip6/tmean_ssp/tmean_'+mod+'_sspfuture_regridcon.nc')#.rename({'lat':'latitude','lon':'longitude'})
    dat1=dat.convert_calendar('noleap').rename({'lon':'longitude','lat':'latitude'})
    temp_dat=rotlon_180(dat1)
    temp_datcut[mod]=temp_dat#.sel(time=(slice('1950-01-01','2024-12-31')))
#     to_con.append(temp_datcut[mod])

In [ ]:
#tempful=xr.concat(to_con, dim='models')
tempful.tas.to_dataset(name='tas').to_netcdf('cmip6/future/tas_day_models_future_ssp5_regridcon.nc')

In [ ]:
#tempful=xr.open_dataset('tas_day_models_future_ssp5_regridcon.nc')

In [ ]:
#compute hotspells
# models_out=['CESM2-WACCM', 'CMCC-CM2-SR5', 'CNRM-CM6-1','HadGEM3-GC31-LL', 'HadGEM3-GC31-MM','INM-CM4-8','IPSL-CM6A-LR', 'MIROC-ES2L','CNRM-CM6-1-HR']

for i,mod in enumerate(models):
    temp=temp_datcut[mod].tas.to_dataset(name='t2m')
    dmaxanom=temp.groupby('time.dayofyear')-temp.groupby('time.dayofyear').mean(dim='time')
    dmaxanom=dmaxanom.sortby(dmaxanom.time)
    #dmaxanom=cuttimestart(dmaxanom,1979)
    outqt_anom=refqtl_single(dmaxanom.drop_vars('dayofyear'))
    #outqt_anom.to_netcdf('cmip6/'+mod+'_anom_quantiles_15dsmooth_1950_2023_1x1.nc')

    thresh_anom=thresh_single(dmaxanom.drop_vars('dayofyear'),outqt_anom)
    thresh_anom=thresh_anom.sortby(thresh_anom.time)
    #thresh_anom.to_netcdf('cmip6/'+mod+'_anom_thresholds_5dsmooth_1950_2023_1x1.nc')

    heat_1_d=heat_day1(thresh_anom,dmaxanom,pst=3)
    heat_1_d.to_dataset(name='htdays').to_netcdf('cmip6/heat/'+models_out[i]+'_tmean_heatdays_future_ssp5_2025_2100_regridcon.nc')

In [ ]:
tempful=xr.open_dataset('cmip6_tas_day_hist_ssp_models_regridcon.nc')
tempful=tempful.sortby(tempful.time)

In [ ]:
#compute hotspells detrended
# detanom=dat_5yanom.to_dataset(name='t2m')
# detanom=detanom.assign_coords(models=detanom.models)
for i,mod in enumerate (models):
    tempful=temp_datcut[mod]#.tas.to_dataset(name='t2m')
#     dmaxanom=temp.groupby('time.dayofyear')-temp.groupby('time.dayofyear').mean(dim='time')
    yrs=np.unique(tempful.time.dt.year.values)
    dat_5yanom,dat_5yclims=rem_5yclim(tempful.tas,30,yrs)
    dmaxanom=dat_5yanom.to_dataset(name='t2m')
    #dmaxanom=cuttimestart(dmaxanom,1979)
    outqt_anom=refqtl_single(dmaxanom.drop_vars('dayofyear'))
    #outqt_anom.to_netcdf('cmip6/'+mod+'_anom_quantiles_15dsmooth_1950_2023_1x1.nc')

    thresh_anom=thresh_single(dmaxanom.drop_vars('dayofyear'),outqt_anom)
    thresh_anom=thresh_anom.sortby(thresh_anom.time)
    #thresh_anom.to_netcdf('cmip6/'+mod+'_anom_thresholds_5dsmooth_1950_2023_1x1.nc')

    heat_1_d=heat_day1(thresh_anom,dmaxanom,pst=3)
    heat_1_d.to_dataset(name='htdays').to_netcdf('cmip6/heat/'+mod+'_detrendedtmean_heatdays_hist_ssp5_1950_2024_regridcon.nc')

In [ ]:
for i,mod in enumerate (models):
    heat_1_d=xr.open_dataset('cmip6/heat/'+mod+'_detrendedtmean_heatdays_hist_ssp5_1950_2024_regridcon.nc')
    heat_1_d.to_netcdf('cmip6/heat/'+models_out[i]+'_detrendedtmean_heatdays_hist_ssp5_1950_2024_regridcon.nc')